In [1]:
import urllib
import requests
import pandas
from selenium import webdriver
import numpy as np
import json
from bs4 import BeautifulSoup
from csv import reader

In [2]:
filenames, urls = [], []
with open("Model Organisms - Sheet1.csv", "r") as f:
    file = reader(f)
    for f, u in file:
        filenames.append(f+".json")
        urls.append(u)

In [3]:
driver=webdriver.Safari()
failed = []
for url,filename in zip(urls,filenames):
    try:
        driver.get(url)
        content=driver.page_source
        soup=BeautifulSoup(content)
        table=[]
        for d in soup.findAll("pre"):
            table.append(d)
        table=str(table[0]).strip("<pre>").strip("\n</")
        split={
            "codon":[],
            "aminoacid":[],
            "frequency":[]
        }
        for i,t in enumerate(table.split()):
            if (i)%6==0:
                split["codon"].append(t)
            elif (i)%6==1:
                split["aminoacid"].append(t)
            elif (i)%6==2:
                split["frequency"].append(t)
        aminoacid,codon=[],[]
        for c in np.unique(split["aminoacid"]):
            common=(np.asarray(split["aminoacid"]))
            common=np.where(common==c)[0]
            index=common
            common=np.array(split["frequency"])[common]
            common=split["codon"][index[common.astype(float).argmax()].astype(int)]
            aminoacid.append(c)
            codon.append(common)
        common=dict(zip(aminoacid,codon))
        with open(filename,"w") as f:
            json.dump(common,f, indent=4, sort_keys=True)
    except ValueError:
        failed.append(filename)
        print(f"{filename} failed to parse")

Dictyostelium discoideum.json failed to parse
Saccharomyces cerevisiae.json failed to parse
Schizosaccharomyces pombe.json failed to parse
Arabidopsis thaliana.json failed to parse
Brachypodium distachyon.json failed to parse
Medicago truncatula.json failed to parse
Oryza sativa.json failed to parse
Caenorhabditis elegans.json failed to parse
Drosophila melanogaster.json failed to parse
Mus musculus.json failed to parse
Rattus norvegicus.json failed to parse
Xenopus laevis.json failed to parse
Gallus gallus.json failed to parse
Danio rerio.json failed to parse
